## Utilisation de la fonction apply de Dataframe
### Calcul de la matrice avec modification du fichier

In [ ]:
# On importe la librairie pandas et le dataset source
import pandas as pd
df = pd.read_csv('storage/batch_prediction_all_fields.csv', index_col=False, header=0)

def seuil(row):
    if row['1 probability'] < seuil_proba:
        seuil = 0
    else: 
        seuil = 1
    return seuil

def confusion(row):
    if(row['SeriousDlqin2yrs'] == 0 and  row['SeriousDlqin2yrs_Predic_Seuil'] == 0):
        confusion_value = 'TN'
    if(row['SeriousDlqin2yrs'] == 1 and  row['SeriousDlqin2yrs_Predic_Seuil'] == 1):
        confusion_value = 'TP'
    if(row['SeriousDlqin2yrs'] == 1 and  row['SeriousDlqin2yrs_Predic_Seuil'] == 0):
        confusion_value = 'FN'
    if(row['SeriousDlqin2yrs'] == 0 and  row['SeriousDlqin2yrs_Predic_Seuil'] == 1):
        confusion_value = 'FP'
    return confusion_value

seuil_proba = 0.01
gain_total_max  = 0
seuil_proba_opti = 0

# Boucle pour déterminer le seuil le plus optimisant
for i in range(0,100):
    df['SeriousDlqin2yrs_Predic_Seuil'] = df.apply(seuil, axis = 1)
    df['Confusion'] = df.apply(confusion, axis = 1)

    erreurs=list(df.Confusion.values)
    tot_TP = erreurs.count('TP')
    tot_TN = erreurs.count('TN')
    tot_FP = erreurs.count('FP')
    tot_FN = erreurs.count('FN')
    total_values = tot_TP + tot_TN + tot_FP + tot_FN
    gain_total = tot_TP * 0 + tot_TN * 500 +  tot_FP * -500 + tot_FN * -2500
    matrice = ['TP : ' + str(tot_TP), 'TN : ' + str(tot_TN), 'FP : ' + str(tot_FP), 'FN : ' + str(tot_FN),]
    
    if gain_total > gain_total_max:
        gain_total_max  = gain_total
        seuil_proba_opti = seuil_proba
        matrice_opti = matrice
    
    print(f"Matrice : {matrice}")
    print(f"Gain total max : {gain_total_max}")
    print(f"Seuil proba utilisé : {round(seuil_proba*100)}%")
    print(f"Seuil proba opti : {round(seuil_proba_opti*100)}%")
    print(f"Nombre total de valeurs : {total_values}")
    print()
    
    seuil_proba += 0.01

# On lance une dernière fois les calculs sur le seuil_proba_opti puis on enregistre le fichier
seuil_proba = seuil_proba_opti - 0.01
df['SeriousDlqin2yrs_Predic_Seuil'] = df.apply(seuil, axis = 1)
df['Confusion'] = df.apply(confusion, axis = 1)
df['Error_abs'] = abs(seuil_proba_opti - df['1 probability'])
df.to_csv('storage/batch_prediction_seuil_optimise.csv', index=False)

print(f"{df.groupby(['Confusion']).Confusion.count()} \n")

df = df[df['Confusion'].isin(["FN", "FP"])]
df.sort_values(by=['Error_abs'], ascending=False, inplace=True)
df.head(100).to_csv('storage/batch_prediction_seuil_optimise_100errors.csv', index=False)

print(f"Résultat Final :")
print(f" >>> Matrice : {matrice_opti}")
print(f" >>> Gain total max : {gain_total_max}")
print(f" >>> Seuil proba opti : {round(seuil_proba_opti*100, 2)}%")
print(f" >>> Accuracy : {round(((tot_TP + tot_TN)/total_values)*100, 2)}%")

In [ ]:
import pandas as pd
df = pd.read_csv('storage/batch_prediction_all_fields.csv', index_col=False, header=0)

def cost_threshold(threshold):
    prediction_threshold = df['1 probability'] > threshold

    #~ = "not" pour les tableaux / & = "and" pour les tableaux / | = "or" pour les tableaux

    P_pred = prediction_threshold
    N_pred = ~prediction_threshold

    P_real = (df['SeriousDlqin2yrs'] == 1)
    N_real = (df['SeriousDlqin2yrs'] == 0)

    TP = len(df.loc[P_pred & P_real])
    TN = len(df.loc[N_pred & N_real])
    FP = len(df.loc[P_pred & N_real])
    FN = len(df.loc[N_pred & P_real])

    TP_cost = 0
    FN_cost = -2500
    FP_cost = -500
    TN_cost = 500

    cost = TP * TP_cost
    cost += TN * TN_cost
    cost += FP * FP_cost
    cost += FN * FN_cost

    return cost

import matplotlib.pyplot as plt
table = [cost_threshold(n/1000) for n in range(0,1000)]
plt.plot(table)

max_val = max(table)
max_threshold = table.index(max_val)/1000
print(f'Le gain maximum est {max_val} et il est obtenu avec un seuil de {round(max_threshold*100,4)}%')

## Utilisation de la fonction loc de Dataframe (méthode Thibaud)
### Calcul de la matrice avec modification du fichier

In [ ]:
# On importe la librairie pandas et le dataset source
# import pandas as pd
# df = pd.read_csv('storage/batch_prediction_all_fields.csv', index_col=False, header=0)

# # Ajoute les données dans la colonne confusion
# df.loc[(df.SeriousDlqin2yrs == 0) & (df.SeriousDlqin2yrs_Predic ==0),'confusion'] = 'TN'
# df.loc[(df.SeriousDlqin2yrs == 1) & (df.SeriousDlqin2yrs_Predic ==1),'confusion'] = 'TP'
# df.loc[(df.SeriousDlqin2yrs == 1) & (df.SeriousDlqin2yrs_Predic ==0),'confusion'] = 'FN'
# df.loc[(df.SeriousDlqin2yrs == 0) & (df.SeriousDlqin2yrs_Predic ==1),'confusion'] = 'FP'

# # Ecrit le fichier
# df.to_csv('storage/kaggle-give-me-credit-train_confusion.csv', index=False)

# # Crée une liste à partir du Dataframe
# Err=list(df.confusion.values)

# # Count les données de la matrice
# ['TP : ' + str(Err.count('TP')),
#  'TN : ' + str(Err.count('TN')),
# 'FP : ' + str(Err.count('FP')),
# 'FN : ' + str(Err.count('FN')),]

## Utilisation de la librairie ConfusionMatrix de pandas_ml
### Calcul de la matrice sans modifier le fichier

In [ ]:
# # On importe la librairie pandas et le dataset source
# import pandas as pd
# df = pd.read_csv('storage/batch_prediction_all_fields.csv', index_col=False, header=0)

# from pandas_ml import ConfusionMatrix
# df = pd.DataFrame(df, columns=['SeriousDlqin2yrs','SeriousDlqin2yrs_Predic'])
# Confusion_Matrix = ConfusionMatrix(df['SeriousDlqin2yrs'], df['SeriousDlqin2yrs_Predic'])
# Confusion_Matrix.print_stats()

# print()
# confusion_matrix = pd.crosstab(df['SeriousDlqin2yrs'], df['SeriousDlqin2yrs_Predic'], rownames=['Actual'], colnames=['Predicted'])
# print (confusion_matrix)

## Conversion du Dataframe en Serie
### Calcul de la matrice sans modifier le fichier

In [ ]:
# # On importe la librairie pandas et le dataset source
# import pandas as pd
# df = pd.read_csv('storage/batch_prediction_all_fields.csv', index_col=False, header=0)

# # Statistiques avec une boucle
# TN = TP = FN = FP = 0
# serie = df.transpose() # here we convert the DataFrame into a Serie
# for ligne in serie:
#     if(serie[ligne][11] == 0 and serie[ligne][13] == 0):
#         TN = TN + 1

#     if(serie[ligne][11] == 1 and serie[ligne][13] == 1):
#         TP = TP + 1

#     if(serie[ligne][11] == 1 and serie[ligne][13] == 0):
#         FN = FN + 1

#     if(serie[ligne][11] == 0 and serie[ligne][13] == 1):
#         FP = FP + 1

# nb_all = TN+TP+FN+FP
# print(f"True Negative :    {TN}    ({round(TN/nb_all*100,4)}%)")
# print(f"True Positive :    {TP}    ({round(TP/nb_all*100,4)}%)")
# print(f"False Negative :   {FN}    ({round(FN/nb_all*100,4)}%)")
# print(f"False Positive :   {FP}    ({round(FP/nb_all*100,4)}%)")